In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

from src.reader import read_lusa, read_timebank

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']


ROOT = Path().resolve().parent.parent

In [2]:
models = [
    'llama2-7b',
    'llama2-7b-chat',
    'llama2-13b',
    'llama2-13b-chat',
    'llama2-70b',
    'llama2-70b-chat',
    'gpt3',
    'chatgpt',
    'gpt4',
]

## Prompt Selection

In [3]:
results_path = ROOT / "results" / "prompt_selection"

In [4]:
df_pt = pd.read_csv(results_path / "portuguese"/ "results.csv")
df_pt["language"] = "Portuguese"

df_en = pd.read_csv(results_path / "english"/ "results.csv")
df_en["language"] = "English"

df = pd.concat([df_pt, df_en])
df.reset_index(inplace=True, drop=True)
df["model"] = pd.Categorical(df.model, ordered=True, categories=models)
df

,model,template,entity,precision,recall,f1,f1_r,language
0,chatgpt,ext,event triggers,0.456522,0.260331,0.331579,0.605356,Portuguese
1,chatgpt,cls,event triggers,0.433511,0.336777,0.379070,0.617449,Portuguese
2,chatgpt,cls_def,event triggers,0.601266,0.392562,0.475000,0.784746,Portuguese
3,chatgpt,ext_def,event triggers,0.612903,0.392562,0.478589,0.782263,Portuguese
4,chatgpt,ext_exp,event triggers,0.490265,0.572314,0.528122,0.638241,Portuguese
...,...,...,...,...,...,...,...,...
355,llama2-7b-chat,cls_def_exp,time expressions,0.065421,0.113821,0.083086,0.138516,English
356,llama2-7b-chat,cls_exp,time expressions,0.067961,0.113821,0.085106,0.118924,English
357,llama2-7b-chat,ext_def_exp,time expressions,0.066390,0.130081,0.087912,0.109416,English
358,llama2-7b-chat,ext_exp,time expressions,0.077206,0.170732,0.106329,0.132826,English


In [5]:
languages = df.language.unique().tolist()
entities = df.entity.unique().tolist()
templates = ["ext", "cls",	"ext_def", "ext_exp", "cls_def", "ext_def_exp",  "cls_exp", "cls_def_exp"]
labels = ["_ _ _", "C _ _", "_ D _", "_ _ E", "C D _", "_ D E",  "C _ E", "C D E"]

In [6]:
data = df[["language", "model", "template", "entity", "f1"]].groupby(
    ["language", "entity", "model", "template"]).mean("f1")
data = data.unstack("template")
data = data[[('f1', "ext"),
            ('f1', "cls"),
            ('f1', "ext_def"),
            ('f1', "ext_exp"),
            ('f1', "cls_def"),
            ('f1', "ext_def_exp"),
            ('f1', "cls_exp"),
            ('f1', "cls_def_exp")]]
(data * 100).round(2)

f1                         \
template                                       ext    cls ext_def ext_exp   
language   entity           model                                           
English    event triggers   llama2-7b         0.00   1.34    0.00   10.30   
                            llama2-7b-chat    6.41   0.00   10.90   28.80   
                            llama2-13b        0.00   1.91    2.93    8.85   
                            llama2-13b-chat   2.28   0.31    2.43   20.90   
                            llama2-70b        8.34   3.31   14.96   26.44   
                            llama2-70b-chat   1.17   3.30    2.29   41.22   
                            gpt3              0.30   3.92    3.04   41.12   
                            chatgpt           8.29  33.93   35.96   55.08   
                            gpt4             20.45  57.82   21.36   72.81   
           participants     llama2-7b          NaN    NaN     NaN     NaN   
                            llama2-7b-chat     NaN    NaN     NaN     NaN   
                            llama2-13b         NaN    NaN     NaN     NaN   
                            llama2-13b-chat    NaN    NaN     NaN     NaN   
                            llama2-70b         NaN    NaN     NaN     NaN   
                            llama2-70b-chat    NaN    NaN     NaN     NaN   
                            gpt3               NaN    NaN     NaN     NaN   
                            chatgpt            NaN    NaN     NaN     NaN   
                            gpt4               NaN    NaN     NaN     NaN   
           time expressions llama2-7b         0.00   0.00    0.00    2.12   
                            llama2-7b-chat    7.07   0.00   11.84   10.63   
                            llama2-13b        1.32   0.00    3.30    2.27   
                            llama2-13b-chat  21.09   8.99   22.22   16.62   
                            llama2-70b       12.26   2.91   15.91   21.93   
                            llama2-70b-chat  32.56  16.04   31.16   25.77   
                            gpt3             37.61  27.01   39.17   37.84   
                            chatgpt          35.82  26.38   30.61   42.64   
                            gpt4             58.45  64.19   53.81   64.22   
Portuguese event triggers   llama2-7b         0.00   0.00    9.30   29.64   
                            llama2-7b-chat    3.49   0.00    9.27   24.20   
                            llama2-13b        0.00   2.57   17.56   33.37   
                            llama2-13b-chat   9.12   4.78   13.27   18.21   
                            llama2-70b        3.76   0.78    7.73   31.75   
                            llama2-70b-chat   2.60  10.84    1.57   30.04   
                            gpt3              2.26  22.42    2.80   50.13   
                            chatgpt          33.16  37.91   47.86   52.81   
                            gpt4              6.25  48.14    5.29   62.73   
           participants     llama2-7b         0.00   1.61    6.18   11.41   
                            llama2-7b-chat   10.57   0.00   13.28   10.43   
                            llama2-13b        0.00   0.00   10.53   21.33   
                            llama2-13b-chat   9.28  13.39    8.96   14.09   
                            llama2-70b        8.53   3.45   11.81   16.51   
                            llama2-70b-chat   5.92  16.43    7.39   22.51   
                            gpt3             16.97  19.48   15.72   27.61   
                            chatgpt          12.10  19.12   13.25   36.91   
                            gpt4              8.06  17.16    7.49   52.04   
           time expressions llama2-7b         2.86   0.00    0.00    0.00   
                            llama2-7b-chat    7.71   5.48   10.03    6.55   
                            llama2-13b        6.45   0.00    4.11    0.00   
                            llama2-13b-chat  33.33   6.55   30.67   22.02   
                            llama2-70b      

## Test

In [21]:
results_path = ROOT / "results" / "test"

In [25]:
df_pt = pd.read_csv(results_path / "portuguese"/ "results.csv")
df_pt["language"] = "Portuguese"

df_en = pd.read_csv(results_path / "english"/ "results.csv")
df_en["language"] = "English"

df = pd.concat([df_pt, df_en])
df.reset_index(inplace=True, drop=True)
df["model"] = pd.Categorical(df.model, ordered=True, categories=models)
df

,model,template,entity,precision,recall,f1,f1_r,language
0,chatgpt,cls_def_exp,event triggers,0.487685,0.571492,0.526273,0.585791,Portuguese
1,gpt3,cls_def_exp,event triggers,0.439163,0.549734,0.488267,0.509444,Portuguese
2,gpt4,cls_exp,event triggers,0.597454,0.666963,0.630298,0.684470,Portuguese
3,llama2-13b,ext_def,event triggers,0.193182,0.098135,0.130153,0.183512,Portuguese
4,llama2-13b-chat,ext_def,event triggers,0.205496,0.076377,0.111363,0.384928,Portuguese
5,llama2-70b,cls_def_exp,event triggers,0.264538,0.309059,0.285071,0.273665,Portuguese
6,llama2-70b-chat,cls,event triggers,0.168570,0.137655,0.151552,0.252957,Portuguese
7,llama2-7b,ext_def,event triggers,0.087687,0.041741,0.056558,0.306139,Portuguese
8,llama2-7b-chat,ext_def,event triggers,0.169525,0.114121,0.136412,0.246640,Portuguese
9,chatgpt,cls_def,participants,0.241742,0.130206,0.169251,0.619661,Portuguese


In [26]:
data = df[["language", "entity", "model", "precision", "recall", "f1", "f1_r"]]

data = data.groupby(["language", "entity", "model"]).mean()
(data * 100).round(2)

precision  recall     f1   f1_r
language   entity           model                                           
English    event triggers   llama2-7b             0.00    0.00   0.00   1.31
                            llama2-7b-chat       15.14    7.81  10.30  24.91
                            llama2-13b           11.40    2.00   3.41  13.01
                            llama2-13b-chat       9.13    2.57   4.02  27.15
                            llama2-70b           32.02   25.36  28.30  26.25
                            llama2-70b-chat      17.37   14.91  16.05  14.24
                            gpt3                  6.89    6.18   6.52  17.88
                            chatgpt              79.38   39.12  52.42  72.09
                            gpt4                 83.31   62.81  71.62  81.95
           participants     llama2-7b              NaN     NaN    NaN    NaN
                            llama2-7b-chat         NaN     NaN    NaN    NaN
                            llama2-13b             NaN     NaN    NaN    NaN
                            llama2-13b-chat        NaN     NaN    NaN    NaN
                            llama2-70b             NaN     NaN    NaN    NaN
                            llama2-70b-chat        NaN     NaN    NaN    NaN
                            gpt3                   NaN     NaN    NaN    NaN
                            chatgpt                NaN     NaN    NaN    NaN
                            gpt4                   NaN     NaN    NaN    NaN
           time expressions llama2-7b             2.36    1.64   1.94   2.10
                            llama2-7b-chat       10.95   18.72  13.81  23.66
                            llama2-13b            3.98    1.37   2.04   5.71
                            llama2-13b-chat      30.65   30.96  30.80  46.71
                            llama2-70b           18.04   19.18  18.59  45.07
                            llama2-70b-chat      34.19   37.53  35.79  55.27
                            gpt3                 42.10   35.53  38.53  64.37
                            chatgpt              55.91   37.17  44.65  70.39
                            gpt4                 58.34   57.81  58.07  77.20
Portuguese event triggers   llama2-7b             8.77    4.17   5.66  30.61
                            llama2-7b-chat       16.95   11.41  13.64  24.66
                            llama2-13b           19.32    9.81  13.02  18.35
                            llama2-13b-chat      20.55    7.64  11.14  38.49
                            llama2-70b           26.45   30.91  28.51  27.37
                            llama2-70b-chat      16.86   13.77  15.16  25.30
                            gpt3                 43.92   54.97  48.83  50.94
                            chatgpt              48.77   57.15  52.63  58.58
                            gpt4                 59.75   66.70  63.03  68.45
           participants     llama2-7b             9.92   13.30  11.36  18.31
                            llama2-7b-chat       19.42    8.98  12.28  47.30
                            llama2-13b            7.24    6.91   7.07  26.39
                            llama2-13b-chat      23.35    7.84  11.74  53.02
                            llama2-70b           19.07   15.53  17.12  34.16
                            llama2-70b-chat      26.17   11.32  15.81  56.76
                            gpt3                 26.69   13.22  17.69  60.35
                            chatgpt              24.17   13.02  16.93  61.97
                            gpt4                 28.18   16.86  21.10  66.18
           time expressions llama2-7b             0.81    2.40   1.21   4.06
                            llama2-7b-chat        6.16   30.80  10.26  12.57
                            llama2-13b            3.10    5.20   3.89   8.78
                            llama2-13b-chat      19.47   35.60  25.18  40.06
                            llama2-70b           22.56   35.20  27.50  39.57
                            llama2-70b-ch